#### Вариант №5

||Первый|Второй|Третий|Четвертый|
|---|---|---|---|---|
|Первый|1|2|3|1|
|Второй|0.5|1|3|1|
|Третий|0.33|0.33|1|0.5|
|Четвертый|1|1|2|1|

In [73]:
import numpy as np

In [74]:
matrix = np.array([
    [1, 2, 3, 1],
    [0.5, 1, 3, 1],
    [0.33, 0.33, 1, 0.5],
    [1, 1, 2, 1]
])

normalized_matrix = matrix / matrix.sum(axis=1, keepdims=True)

print("Нормализованная матрица: \n", normalized_matrix)


Нормализованная матрица: 
 [[0.14285714 0.28571429 0.42857143 0.14285714]
 [0.09090909 0.18181818 0.54545455 0.18181818]
 [0.15277778 0.15277778 0.46296296 0.23148148]
 [0.2        0.2        0.4        0.2       ]]


In [75]:
eigenvalues, eigenvectors = np.linalg.eig(matrix)
max_index = np.argmax(eigenvalues)
principal_eigenvector = eigenvectors[:, max_index]
weights = principal_eigenvector / np.sum(principal_eigenvector)
λmax = max(eigenvalues)

print("λmax:", λmax)
print("Собственные значения:", eigenvalues)
print("Собственные векторы:\n", eigenvectors)
print("Главный собственный вектор:", principal_eigenvector)
print("Вектор весов (приоритетов):", weights)

λmax: (4.076676945835413+0j)
Собственные значения: [ 4.07667695+0.j         -0.00691074+0.57590852j -0.00691074-0.57590852j
 -0.06285547+0.j        ]
Собственные векторы:
 [[ 0.68112955+0.j         -0.73125975+0.j         -0.73125975-0.j
  -0.56687091+0.j        ]
 [ 0.47987697+0.j          0.09529148-0.47933042j  0.09529148+0.47933042j
   0.34046658+0.j        ]
 [ 0.20780645+0.j          0.16483358+0.03174468j  0.16483358-0.03174468j
  -0.26062109+0.j        ]
 [ 0.5124423 +0.j          0.0512296 +0.44228809j  0.0512296 -0.44228809j
   0.70343197+0.j        ]]
Главный собственный вектор: [0.68112955+0.j 0.47987697+0.j 0.20780645+0.j 0.5124423 +0.j]
Вектор весов (приоритетов): [0.36206121+0.j 0.25508339+0.j 0.11046159+0.j 0.27239382+0.j]


##### EM

In [76]:
eigenvalues, eigenvectors = np.linalg.eig(matrix.T)
max_eigenvalue_index = np.argmax(eigenvalues)
weights = np.real_if_close(eigenvectors[:, max_eigenvalue_index].T / np.sum(eigenvectors[:, max_eigenvalue_index]))

print("Веса каналов (метод ЕМ):", weights)

Веса каналов (метод ЕМ): [0.14416284 0.20462238 0.46266194 0.18855284]


##### RGMM

In [77]:
geometric_means = np.power(np.prod(matrix, axis=1), 1/len(matrix))
weights_rgmm = geometric_means / np.sum(geometric_means)

print("Веса каналов (метод RGMM):", weights_rgmm)


Веса каналов (метод RGMM): [0.36028387 0.25475916 0.11120043 0.27375654]


##### AN

In [78]:
eigenvalues_an, eigenvectors_an = np.linalg.eig(matrix.T)
max_eigenvalue_index_an = np.argmax(eigenvalues_an)
weights_an = np.real_if_close(eigenvectors_an[:, max_eigenvalue_index_an].T / np.sum(eigenvectors_an[:, max_eigenvalue_index_an]))

print("Веса каналов (метод AN):", weights_an)

Веса каналов (метод AN): [0.14416284 0.20462238 0.46266194 0.18855284]


##### Линия

In [80]:
preference_values = {
    'A': 0.5,
    'C': 3,
    'D': 1,
}

weight_b = 1 / sum(preference_values.values())

weights_line_method = {}
for alternative in preference_values:
    weight = preference_values[alternative] * weight_b
    weights_line_method[alternative] = weight

total_weight = sum(weights_line_method.values())
normalized_weights_line_method = {alternative: weight / total_weight for alternative, weight in weights_line_method.items()}

normalized_weights_line_method['B'] = weight_b

print("Веса каналов (метод 'линия'):", normalized_weights_line_method)


Веса каналов (метод 'линия'): {'A': 0.11111111111111112, 'C': 0.6666666666666667, 'D': 0.22222222222222224, 'B': 0.2222222222222222}


##### MRCI

In [81]:
def calculate_CI(matrix):
    eigenvalues, _ = np.linalg.eig(matrix)
    max_index = np.argmax(eigenvalues)
    max_eigenvalue = eigenvalues[max_index].real
    n = len(matrix)
    CI = (max_eigenvalue - n) / (n - 1)
    return CI

num_matrices = 10
matrix_size = 4

CI_values = []

for _ in range(num_matrices):
    random_matrix = np.random.randint(1, 10, size=(matrix_size, matrix_size))
    CI = calculate_CI(random_matrix)
    CI_values.append(CI)

MRCI = np.mean(CI_values)

print("Индексы согласованности для сгенерированных матриц:")
for i, CI in enumerate(CI_values, 1):
    print(f"Матрица {i}: CI = {CI}")

print(f"\nСреднее значение индексов согласованности (MRCI): {MRCI}")


Индексы согласованности для сгенерированных матриц:
Матрица 1: CI = 2.3814146926906563
Матрица 2: CI = 5.061826900016203
Матрица 3: CI = 5.863392539432122
Матрица 4: CI = 5.607132139994474
Матрица 5: CI = 5.254664644135521
Матрица 6: CI = 4.910900031562323
Матрица 7: CI = 5.830432378739388
Матрица 8: CI = 4.841594432131781
Матрица 9: CI = 5.163725625081753
Матрица 10: CI = 5.481838687753211

Среднее значение индексов согласованности (MRCI): 5.0396922071537436


$$
CI = \frac{λ_{max} - n}{n - 1}
$$

In [82]:
CI = (λmax - len(matrix)) / (len(matrix) - 1)
print("CI:", CI)

CI: (0.02555898194513754+0j)


$$
CR = \frac{CI}{MRCI}
$$

In [83]:
CR = CI/MRCI

print("Отношение согласованности CR:", CR)

Отношение согласованности CR: (0.005071536295184272+0j)


$$
GCI = \frac{λ_{max} - n}{n - 1}
$$

In [84]:
GCI = (λmax - len(matrix)) / (len(matrix) - 1)
print("Геометрический индекс согласованности GCI:", GCI)

Геометрический индекс согласованности GCI: (0.02555898194513754+0j)


##### HM(s)

In [85]:
def calculate_hm(row):
    n = len(row)
    harmonic_sum = sum(1 / element for element in row)
    hm = n / harmonic_sum
    return hm

HMs = [calculate_hm(row) for row in normalized_matrix]

for i, hm in enumerate(HMs):
    print(f"Гармоническое среднее для строки {i + 1}: {hm}")

Гармоническое среднее для строки 1: 0.3479773814702044
Гармоническое среднее для строки 2: 0.24257125530624618
Гармоническое среднее для строки 3: 0.10610932475884245
Гармоническое среднее для строки 4: 0.2638522427440633


$$
HCI(n) = \frac{(HM(s) - n)(n + 1)}{n(n-1)}
$$

In [89]:
n = matrix.shape[0]
RI = {3: 0.58, 4: 0.9, 5: 1.12, 6: 1.24, 7: 1.32, 8: 1.41, 9: 1.45, 10: 1.49}[n]
HCI = CI / RI

print("Гармонический индекс согласованности (HCI):", HCI)

Гармонический индекс согласованности (HCI): (0.0283988688279306+0j)


##### HRCI(n)

$$
HCR(n)=\frac{HCI(n)}{HRCI(n)}
$$

In [90]:
HCR = HCI / 0.89
print("Гармонические отношения согласованности HCR:", HCR)

Гармонические отношения согласованности HCR: (0.03190884137969731+0j)


$$
\begin{align*}
k_c &= \min_{h=1,\dots,5} k_c(R_h) 
\end{align*}
$$

In [91]:
def calculate_spectrum(matrix):
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    max_index = np.argmax(eigenvalues)
    principal_eigenvector = eigenvectors[:, max_index]
    return principal_eigenvector / np.sum(principal_eigenvector)

n_matrices = 5

spectra = []
for _ in range(n_matrices):
    random_matrix = np.random.rand(normalized_matrix.shape[0], normalized_matrix.shape[1])
    random_matrix /= random_matrix.sum(axis=1)[:, np.newaxis]
    spectrum = calculate_spectrum(random_matrix)
    spectra.append(spectrum)

kc_values = [np.min(kc) for kc in spectra]
kc = min(kc_values)

print("Спектральный коэффициент согласованности (kc):", kc)

Спектральный коэффициент согласованности (kc): (0.24999999999999983-0j)


In [172]:
matrix = np.array([
    [1, 0.5, 0.25, 2, 1, 0.5, 2],
    [2, 1, 0.25, 0.5, 1, 0.5, 2],
    [4, 4, 1, 2, 5, 2, 0.142],
    [0.5, 2, 0.5, 1, 2, 1, 4],
    [1, 1, 0.2, 0.5, 1, 0.5, 2],
    [2, 2, 0.5, 1, 2, 1, 4],
    [0.5, 0.5, 7, 0.25, 0.5, 0.25, 1]])

normalized_matrix = matrix / matrix.sum(axis=1, keepdims=True)
print("Нормализованная матрица:\n", normalized_matrix)

Нормализованная матрица:
 [[0.13793103 0.06896552 0.03448276 0.27586207 0.13793103 0.06896552
  0.27586207]
 [0.27586207 0.13793103 0.03448276 0.06896552 0.13793103 0.06896552
  0.27586207]
 [0.22048286 0.22048286 0.05512071 0.11024143 0.27560357 0.11024143
  0.00782714]
 [0.04545455 0.18181818 0.04545455 0.09090909 0.18181818 0.09090909
  0.36363636]
 [0.16129032 0.16129032 0.03225806 0.08064516 0.16129032 0.08064516
  0.32258065]
 [0.16       0.16       0.04       0.08       0.16       0.08
  0.32      ]
 [0.05       0.05       0.7        0.025      0.05       0.025
  0.1       ]]


In [177]:
eigenvalues, _ = np.linalg.eig(matrix)
λmax = max(eigenvalues)
n = len(matrix)
threshold = 0.1
CI = (λmax - n)/(n - 1)
print(f"CI = {CI}")
CR = CI / 1.32
print(f"CR = {CR}")
if CR > threshold:
    print("МПС несогласована!")
    

CIs = []
for i in range(len(matrix)):
    m_copy = np.copy(matrix)
    eigenvalues, _ = np.linalg.eig(np.delete(np.delete(matrix, i, axis=0), i, axis=1))
    λmax = max(eigenvalues)
    n = len(matrix)
    CIs.append(((λmax - n)/(n - 1), i))
    print(f"{i + 1} CI = {CIs[i][0]}")

min_CIs = sorted(CIs)[:2]
print(f"Выброс: [{min_CIs[0][1] + 1}, {min_CIs[1][1] + 1}] = {matrix[min_CIs[1][1]][min_CIs[0][1]]}")


CI = (0.5230140224026594+0j)
CR = (0.39622274424443893+0j)
МПС несогласована!
1 CI = (0.3172765558444972+0j)
2 CI = (0.3490111768319502+0j)
3 CI = (-0.10735911872128619+0j)
4 CI = (0.32458515077366956+0j)
5 CI = (0.34508108797220505+0j)
6 CI = (0.362099097740607+0j)
7 CI = (-0.10649746567621943+0j)
Выброс: [3, 7] = 7.0


In [188]:
D = normalized_matrix


correlation_matrix = np.corrcoef(D)
M_Rir = np.mean(D, axis=1)

M_Ric = np.mean(D, axis=0)

min_index_Rir = np.argmin(M_Rir)
min_index_Ric = np.argmin(M_Ric)

outlier_row = min_index_Rir + 1
outlier_column = min_index_Ric + 1

print(f"Матрица коэффициентов корреляции:\n{correlation_matrix}\n")
print(f"Элемент d{outlier_row}{outlier_column} (и соответственно d{outlier_column}{outlier_row}) МПС D – выброс.")

Матрица коэффициентов корреляции:
[[ 1.          0.42690058 -0.29616234  0.51507269  0.5511879   0.54437945
  -0.44768943]
 [ 0.42690058  1.          0.09768303  0.51507269  0.85105697  0.84923193
  -0.41221215]
 [-0.29616234  0.09768303  1.         -0.2600425  -0.09370505 -0.1082304
  -0.42761568]
 [ 0.51507269  0.51507269 -0.2600425   1.          0.88778812  0.89000667
  -0.29759498]
 [ 0.5511879   0.85105697 -0.09370505  0.88778812  1.          0.99960413
  -0.42819028]
 [ 0.54437945  0.84923193 -0.1082304   0.89000667  0.99960413  1.
  -0.40259536]
 [-0.44768943 -0.41221215 -0.42761568 -0.29759498 -0.42819028 -0.40259536
   1.        ]]

Элемент d16 (и соответственно d61) МПС D – выброс.
